In [1]:
import os 
import sys
import numpy as np 
import pandas as pd

from pathlib import Path
from copy import deepcopy

sys.path.append('yolov7')

def moving_average(interval, min_kernel_size=5, max_kernel_size = 99):
    kernel_size = len(interval) // 4 * 2 - 1
    
    if kernel_size < min_kernel_size:
        ans = np.ones(len(interval)) * int(np.average(interval))
    else:
        kernel_size = min(max_kernel_size, kernel_size)
        window = np.ones(int(kernel_size)) / float(kernel_size)
        re = np.convolve(interval, window, 'valid').astype(np.uint16)
        '''
        平滑后输出维度不等于输入维度，使用'same'参数，但仍存在边缘值问题，参考https://gemini-yang.blog.csdn.net/article/details/107176500
        因此，使用'valid'参数获取可用结果，然后对结果的边界进行等值填充
        填充范围为 (kernel_size-1) //2, kernel_size需要为奇数'''
        lb = np.ones((kernel_size-1)//2)*re[0]
        rb = np.ones((kernel_size-1)//2)*re[-1]
        ans = np.concatenate((lb, re, rb))
        # print(kernel_size, len(interval), len(ans))

    return ans

def change_bbox(det_dict, kernel_size = 13):
    smooth_dict = {}
    bboxs = []
    for key, data in det_dict.items():
        bboxs.append(data[-4:])
    bboxs = np.array(bboxs)
    for i in range(4):
        bboxs[:, i] = moving_average(bboxs[:, i], kernel_size)
    for i, key in det_dict.keys():
        smooth_dict[key] = bboxs[i]
        
    return smooth_dict

# 传统txt读取（已废弃）

In [2]:
txt_path = './runs/track/1629190984540.txt'
with open(txt_path, 'r') as fr:
    lines = fr.readlines()
track_info = {}

for line in lines:
    line_list = line.strip().split(' ')
    line_list = [float(tmp) for tmp in line_list]
    video_id, frame_id, track_id, cls_id, conf, x, y, w, h = line_list
    if track_id not in track_info:
        track_info[int(track_id)] = {int(frame_id): [int(cls_id), conf, x, y, w, h]}
    else:
        track_info[int(track_id)][int(frame_id)] = [int(cls_id), conf, x, y, w, h]
        
print(len(track_info))

21


# pandas读取

In [3]:
def change_bbox_pandas(det_df, kernel_size = 13):
    # det_df为按照track_id 分组后的单一 track id
    smooth_dict = {}
    smooth_df = deepcopy(det_df).reset_index(drop=True)
    for col in ['x', 'y', 'w', 'h']:
        # print(len(np.array(det_df[col])), len(smooth_df[col]))
        smooth_df.drop(col, axis=1, inplace=True)
        size = len(det_df[col])
        smooth_npy = moving_average(np.array(det_df[col]), min_kernel_size=5, max_kernel_size = 13)
        smooth_dict[col] = smooth_npy
    
    smooth_df = pd.concat([smooth_df, pd.DataFrame(smooth_dict)], axis=1)
    return smooth_dict, smooth_df

In [4]:


txt_path = './runs/track/1629190984540.txt'
txt_columns = ['video_id', 'frame_id', 'track_id', 'cls_id', 'conf', 'x', 'y', 'w', 'h', 'not_use']

df = pd.read_csv(txt_path, sep=' ', header=None)
df.columns = txt_columns
df.drop('not_use',axis=1, inplace=True)
track_df = df.groupby('track_id', axis = 0, as_index = False)

ans_df = pd.DataFrame(columns = txt_columns[:-1])
for name, data in track_df:
    smooth_dict, smooth_df = change_bbox_pandas(data)
    ans_df = pd.concat([ans_df, smooth_df], axis = 0)

ans_df.sort_values(by = ['video_id', 'frame_id'], inplace = True)
ans_df.to_csv(os.path.splitext(txt_path)[0]+'_smooth.txt', sep = ' ', header=None, index=False)

# videos

In [6]:
import cv2
import random
from yolov7.utils.plots import plot_one_box, color_list

names = {0:'logo', 1:'ren', 2:'shiwu', 3:'wenzi'}
colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]

video_path = '../dataset/videos0601/'
video_name = '1629190984540.mp4'
save_path = './runs/track/'

txt_path = './runs/track/1629190984540_smooth.txt'
txt_columns = ['video_id', 'frame_id', 'track_id', 'cls_id', 'conf', 'x', 'y', 'w', 'h']
df = pd.read_csv(txt_path, sep=' ', header=None)
df.columns = txt_columns

frame_index = 0
print(os.path.join(video_path, video_name))
cap = cv2.VideoCapture(os.path.join(video_path, video_name))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))# 获取视频宽度
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))# 获取视频高度
fps = cap.get(cv2.CAP_PROP_FPS) #视频平均帧率
print(fps, frame_width, frame_height)

# 设置输出视频的参数，如果是灰度图，可以加上 isColor = 0 这个参数
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')  # 设置输出视频为mp4格式
write_video = cv2.VideoWriter(os.path.join(save_path, os.path.splitext(video_name)[0]+'_smooth.mp4'),\
                              fourcc, fps, (frame_width,frame_height) ) #设置保存视频的名称和路径

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break
    
    frame_df = df.loc[df['frame_id'] == frame_index]
    frame_df.reset_index(drop=True, inplace=True)
    
    for i in range(len(frame_df)):
        x, y, w, h = frame_df['x'][i], frame_df['y'][i],frame_df['w'][i], frame_df['h'][i]
        x1, y1 = int(x - w/2), int(y - h/2)
        x2, y2 = int(x + w/2), int(y + h/2)
        bboxes = [x1, y1, x2, y2]
        
        track_id = frame_df['track_id'][i]
        cls_id = frame_df['cls_id'][i]
        conf = frame_df['conf'][i]
        label = f"{track_id} {names[cls_id]}:{conf:.2f}"
        plot_one_box(bboxes, img, label=label, color=colors[cls_id], line_thickness=3)    
    write_video.write(img)
    frame_index += 1
    
cap.release()  # 释放读取mp4
write_video.release() 

../dataset/videos0601/1629190984540.mp4
25.0 720 1280
